In [ ]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split

import io
import bson # this is installed with the pymongo package
import matplotlib
import matplotlib.pyplot as plt
from skimage.data import imread   # or, whatever image library you prefer
import multiprocessing as mp      # will come in handy due to the size of the data
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from collections import Counter
from keras.utils import np_utils
from tflearn.data_utils import to_categorical

from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression

import tensorflow as tf
import tflearn

In [ ]:
def getRawFeatures(picture):
    red = []
    green = []
    blue = []
    for row in range(picture.shape[0]):
        for col in range(picture.shape[1]):
            red.append(picture[row][col][0])
            green.append(picture[row][col][1])
            blue.append(picture[row][col][2])
    feature = red
    feature.extend(green)
    feature.extend(blue)
    return feature

In [ ]:
# Simple data processing
count_images = 0
image_names_array = []
category_id_array = []
data = bson.decode_file_iter(open('../input/train_example.bson', 'rb'))
pictures = []
count = 0
prod_to_category = dict()

for c, d in enumerate(data):
    #for each product_id
    product_id = d['_id']
    category_id = d['category_id'] # This won't be in Test data
    prod_to_category[product_id] = category_id
    
    for e, pic in enumerate(d['imgs']):
        #for each image
        picture = imread(io.BytesIO(pic['picture']))
        pictures.append(picture)
        count = count + 1
        # do something with the picture, etc
#         image_name = "prod_id-" + str(product_id) + "-" + "image-" + str(e)
#         print("PRODUCT ID:", product_id, "NUMBER", e)
#         plt.imshow(picture)
#         fig1 = plt.gcf()
#         plt.show()
#         plt.draw()
        count_images = count_images + 1
#         image_names_array.append(image_name)
        category_id_array.append(str(category_id))
        #fig1.savefig("img/" + str(image_name), dpi=100)
#     break

prod_to_category = pd.DataFrame.from_dict(prod_to_category, orient='index')
prod_to_category.index.name = '_id'
prod_to_category.rename(columns={0: 'category_id'}, inplace=True)

In [ ]:
X = np.asarray(pictures)
y = np.asarray(category_id_array)

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
X = X.reshape(X.shape[0], 3, 180, 180).astype('float32')
X = X - np.mean(X) / X.std()

In [ ]:
y

In [ ]:
b,c = np.unique(y, return_inverse=True)

In [ ]:
Counter(c)
y = c
y

In [ ]:
y = np_utils.to_categorical(y)

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
#lets split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1234)
X_train = X
y_train = y

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
model = input_data(shape=[None,3,180,180])
model = conv_2d(model,180,10,activation='elu')
model = max_pool_2d(model,2)
model = conv_2d(model,360,5,activation='relu')
model = max_pool_2d(model,2)
model = dropout(model,0.3)
model = conv_2d(model,360,5, activation='elu')
model = max_pool_2d(model,2)
model = fully_connected(model,1024,activation='sigmoid')
model = dropout(model,0.3)
model = fully_connected(model,512,activation='sigmoid')
model = dropout(model,0.3)
model = fully_connected(model,36,activation='softmax')
model = regression(model,optimizer='adagrad',loss='categorical_crossentropy',learning_rate=0.05)

In [ ]:
with tf.device('cpu:0'):
   model = tflearn.DNN(model)
   model.fit(X_train , y_train, n_epoch=5, validation_set = (X_test, y_test), batch_size = 10)

In [ ]:
model.evaluate(X_test, y_test)